# ``jenga`` Corruptions

This notebook illustrates some of the corruptions used in jenga

Most corruptions are operating on tabular data, for which different sampling mechanisms are used. 

Inspired by research on missing values, we consider three different sampling schemes for corrupting tabular data (we'll keep using the 'missingness' terminology here, even if we use other corruptions than discarding data):

* **MCAR** Missing completely at random
* **MAR** Missing at random - corruption is conditioned on other column
* **MNAR** Missing not at random - corruption is conditioned on values in column on which it is applied


In [1]:
%load_ext autoreload
%autoreload 2

import random
import pandas as pd
import numpy as np

from jenga.corruptions.generic import MissingValues

In [2]:
def new_df(N=20):
    return pd.DataFrame({
        'A':np.arange(0,N) / N, 
        'B': [random.choice(['a','b','c']) for _ in range(N)],
        'C': [random.choice(['foo','bah']) for _ in range(N)]
        })

## Missing Values

Below some examples on different row sampling strategies for corruptions with missing values

In [3]:
from jenga.corruptions.generic import MissingValues

df = new_df()

df['B_MCAR'] = MissingValues(column='B', fraction=.5, missingness='MCAR').transform(df)['B']
df['B_MAR'] = MissingValues(column='B', fraction=.5, missingness='MAR').transform(df)['B']
df['B_MNAR'] = MissingValues(column='B', fraction=.5, missingness='MNAR').transform(df)['B']

df

,A,B,C,B_MCAR,B_MAR,B_MNAR
0,0.00,a,bah,a,a,a
1,0.05,b,bah,b,b,b
2,0.10,a,bah,NaN,NaN,a
3,0.15,a,bah,NaN,NaN,a
4,0.20,b,foo,b,b,NaN
5,0.25,c,bah,NaN,NaN,c
6,0.30,a,foo,a,a,NaN
7,0.35,c,bah,NaN,NaN,c
8,0.40,b,foo,NaN,b,NaN
9,0.45,b,bah,b,NaN,NaN


If we sort according to the values in the column on which the corruption as applied, we see that the 'MNAR' condition works as it should

In [4]:
df.sort_values('B')

,A,B,C,B_MCAR,B_MAR,B_MNAR
0,0.00,a,bah,a,a,a
2,0.10,a,bah,NaN,NaN,a
3,0.15,a,bah,NaN,NaN,a
15,0.75,a,foo,NaN,a,NaN
6,0.30,a,foo,a,a,NaN
18,0.90,a,foo,NaN,a,NaN
17,0.85,b,foo,b,NaN,NaN
16,0.80,b,foo,b,NaN,NaN
14,0.70,b,bah,b,NaN,NaN
13,0.65,b,foo,b,NaN,NaN


## Swapping Values

Also works between numeric and non-numeric values, with different sampling schemes

In [5]:
from jenga.corruptions.generic import SwappedValues
df = new_df()
df['C_swapped_MAR'] = SwappedValues(column='B', fraction=.5, sampling='MAR').transform(df)['C']
df['C_swapped_MCAR'] = SwappedValues(column='B', fraction=.5, sampling='MCAR').transform(df)['C']
df['A_swapped_MCAR'] = SwappedValues(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,C_swapped_MAR,C_swapped_MCAR,A_swapped_MCAR
0,0.00,b,foo,b,b,0.0
1,0.05,a,bah,a,a,0.05
2,0.10,a,bah,a,a,0.1
3,0.15,b,foo,b,foo,foo
4,0.20,c,bah,c,c,c
5,0.25,a,foo,a,a,a
6,0.30,a,bah,a,a,a
7,0.35,b,bah,b,b,b
8,0.40,a,bah,a,bah,0.4
9,0.45,b,bah,b,bah,bah


## Messing up categorical Values

These corruptions permute the histogram of values

In [6]:
from jenga.corruptions.generic import CategoricalShift
df = new_df()
df['C_permuted_MAR'] = CategoricalShift(column='B', fraction=.5, sampling='MAR').transform(df)['C']
df['C_permuted_MCAR'] = CategoricalShift(column='B', fraction=.5, sampling='MCAR').transform(df)['C']
df['A_permuted_MCAR'] = CategoricalShift(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

CategoricalShift implemented only for categorical variables


,A,B,C,C_permuted_MAR,C_permuted_MCAR,A_permuted_MCAR
0,0.00,a,foo,foo,foo,0.00
1,0.05,b,bah,bah,bah,0.05
2,0.10,b,foo,foo,foo,0.10
3,0.15,b,foo,foo,foo,0.15
4,0.20,c,foo,foo,foo,0.20
5,0.25,c,bah,bah,bah,0.25
6,0.30,b,foo,foo,foo,0.30
7,0.35,c,bah,bah,bah,0.35
8,0.40,c,foo,foo,foo,0.40
9,0.45,c,foo,foo,foo,0.45


## Adding Gaussian Noise



In [7]:
from jenga.corruptions.numerical import GaussianNoise
df = new_df()
df['A_GaussNoise_MAR'] = GaussianNoise(column='A', fraction=.5, sampling='MAR').transform(df)['A']
df['A_GaussNoise__MCAR'] = GaussianNoise(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,A_GaussNoise_MAR,A_GaussNoise__MCAR
0,0.00,b,bah,0.000000,0.000000
1,0.05,a,foo,-1.127466,0.050000
2,0.10,b,foo,-1.233787,0.813063
3,0.15,c,foo,0.274392,0.150000
4,0.20,c,bah,0.200000,0.200000
5,0.25,c,foo,-0.212652,0.250000
6,0.30,c,bah,0.300000,2.323479
7,0.35,a,foo,0.441083,0.350000
8,0.40,c,bah,0.400000,-0.045074
9,0.45,a,foo,0.011400,0.450000


## Scaling numeric values


In [8]:
from jenga.corruptions.numerical import Scaling
df = new_df()
df['A_Scaled_MAR'] = Scaling(column='A', fraction=.5, sampling='MAR').transform(df)['A']
df['A_Scaled_MCAR'] = Scaling(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,A_Scaled_MAR,A_Scaled_MCAR
0,0.00,b,foo,0.00,0.00
1,0.05,a,foo,50.00,0.50
2,0.10,a,foo,100.00,0.10
3,0.15,c,foo,150.00,0.15
4,0.20,b,foo,200.00,2.00
5,0.25,b,foo,250.00,0.25
6,0.30,b,foo,300.00,0.30
7,0.35,c,foo,350.00,0.35
8,0.40,a,foo,0.40,4.00
9,0.45,b,foo,450.00,4.50
